# Web scraping Otomoto
The program that collects data from the Otodom website (https://www.otomoto.pl/) and saves in a csv file and access to the data

In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2
import json

Load data from json

In [3]:
with open('Data_to_Otomoto.json') as file:
    Data_to_Otomoto = json.loads(file.read())

insert_table = Data_to_Otomoto["insert_table"]
car_brand = Data_to_Otomoto["car_brand"]
table_cars = Data_to_Otomoto["table_cars"]
CREATE_TABLE = Data_to_Otomoto["CREATE_TABLE"]

Addition of the euro exchange rate

In [4]:
html_data_eur = requests.get("https://www.bankier.pl/waluty/kursy-walut/forex/eurpln")
soup = BeautifulSoup(html_data_eur.content, 'html5lib')
eur = soup.find('div',attrs={"class":"profilLast"}).text
eur = eur.replace(",",".")
print(eur)

4.6330 


Adding a function that retrieves the price clears the data, if the price is in EUR, it converts it to PLN

In [5]:
def get_price(soup):
    Cena = soup.find('span',attrs={"class":"offer-price__number"}).text
    Cena = Cena.replace(" ","").replace(",",".")
    if "PLN" in Cena:
        Cena = Cena.replace("PLN","")
        Cena = float(Cena)
    elif "EUR" in Cena:
        Cena = Cena.replace("EUR","")
        Cena = float(Cena)
        Cena = Cena * eur
    else:
        Cena = np.NaN
    return Cena

A function that gets the ID of the advertisement

In [6]:
def take_id(soup):
    ID = soup.find('div',attrs={"class":"offer-content__metabar"})
    ID = ID.find('div')
    ID = ID.find('span',attrs={"id":"ad_id"}).text
    return ID

In [7]:
def take_localization(soup):
    try:
        localization = soup.find('a',attrs={"class":"seller-card__links__link__cta"}).text.strip()
        locator = Nominatim(user_agent="myGeocoder") 
        location = locator.geocode(localization)
        latitude = location.latitude
        longitude = location.longitude
    except:
        localization = np.nan
        latitude = np.nan
        longitude = np.nan
        
    return localization, latitude, longitude

In [8]:
html_data = requests.get("https://www.otomoto.pl/oferta/mercedes-benz-gla-serwisowany-aso-polski-salon-prywatnie-ID6DXUZi.html")
soup = BeautifulSoup(html_data.content, 'html5lib')
take_localization(soup)

('Poznań, Wielkopolskie, Stare Miasto', 52.4087213, 16.9336538)

Getting details and cleaning them. Then save the data to the dictionary.

In [9]:
def take_details(soup):
    details = {}    
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip().replace(" ","_").replace("(","").replace(")","")
        key = key.replace("ś","s").replace("ż","z").replace("ó","o").replace("ć","c").replace("ę","e")
        key = key.replace("ł","l").replace("ą","a")
        if key in insert_table:
            value = info.find('div').text.strip()
            details[key] = value
    return details

Combine all data into one dictionary

In [10]:
def take_all(soup):
    dict_all = {}
    dict_all["ID"] = take_id(soup)
    dict_all["Cena"] = get_price(soup)
#         dict_all["localization","latitude","longitude"] = take_localization(soup)
    texts = ["localization","latitude","longitude"]
    for (x, y) in zip(texts, take_localization(soup)): 
        dict_all[x] = y
    details = take_details(soup)
    return dict_all | details

In [11]:
take_all(soup)

{'ID': '6087354512',
 'Cena': 75000.0,
 'localization': 'Poznań, Wielkopolskie, Stare Miasto',
 'latitude': 52.4087213,
 'longitude': 16.9336538,
 'Oferta_od': 'Osoby prywatnej',
 'Kategoria': 'Osobowe',
 'Marka_pojazdu': 'Mercedes-Benz',
 'Model_pojazdu': 'GLA',
 'Wersja': '250 4-Matic',
 'Rok_produkcji': '2014',
 'Przebieg': '176 000 km',
 'Pojemnosc_skokowa': '1 991 cm3',
 'Rodzaj_paliwa': 'Benzyna',
 'Moc': '211 KM',
 'Skrzynia_biegow': 'Automatyczna',
 'Naped': '4x4 (stały)',
 'Spalanie_W_Miescie': '8 l/100km',
 'Typ_nadwozia': 'SUV',
 'Emisja_CO2': '154 g/km',
 'Liczba_drzwi': '5',
 'Liczba_miejsc': '5',
 'Kolor': 'Niebieski',
 'Rodzaj_koloru': 'Metalik',
 'Kraj_pochodzenia': 'Polska',
 'Numer_rejestracyjny_pojazdu': 'PO7LS08',
 'Zarejestrowany_w_Polsce': 'Tak',
 'Serwisowany_w_ASO': 'Tak',
 'Stan': 'Używane'}

Data cleaning

In [12]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'][:-2].replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'][:-2].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Pojemnosc_skokowa'] = details['Pojemnosc_skokowa'][:-3].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Miescie'] = details['Spalanie_W_Miescie'][:-7].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Cyklu_Mieszanym'] = details['Spalanie_W_Cyklu_Mieszanym'][:-7].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_Poza_Miastem'] = details['Spalanie_Poza_Miastem'][:-7].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Emisja_CO2'] = details['Emisja_CO2'][:-4].replace(" ","").replace(",",".")
    except:
        pass
    return details

In [13]:
type_list = ['Cena','Moc','Pojemnosc_skokowa','Liczba_drzwi','Przebieg','Rok_produkcji',
             'Spalanie_W_Miescie','Emisja_CO2','Liczba_miejsc','Spalanie_W_Cyklu_Mieszanym',
             'Spalanie_Poza_Miastem']

In [14]:
table_cars = pd.DataFrame(table_cars)

## Main loop

In [ ]:
Link_ERROR = 0
Read_announcements = 0
new_row = {}
all_data = []
for brand in car_brand:
    brand_link = f"https://www.otomoto.pl/osobowe/{brand}/?search%5Border%5D=created_at%3Adesc&page="
    for page_main in range(1,501): 
        try:
            html_data_link = requests.get(brand_link + str(page_main))
            if brand in html_data_link.url:       
                soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
                links = []
                for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
                    if "https://www.otomoto.pl" in offer['href']:
                        links.append(offer['href']) 
                for page in links:
                    print(page)
                    html_data = requests.get(page)
                    soup = BeautifulSoup(html_data.content, 'html5lib')
                    dict_all = take_all(soup)
                    dict_all = data_clean(dict_all)
                    dict_all["Link"] = page
                    if dict_all:
                        all_data.append(dict_all)
                    Read_announcements += 1
                    clear_output(wait=True)
                    print("Read_announcements ", Read_announcements) 
        except Exception:
            print("Link_ERROR")
            continue
table_cars = table_cars.append(all_data, ignore_index=True)
#table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
delete_duplicates = Read_announcements - len(table_cars.index) 
print("Delete duplicates: ", delete_duplicates)
print("Finish")

Read_announcements  26624
https://www.otomoto.pl/oferta/bmw-x1-sdrive-18d-xline-150km-navi-oplacony-ID6E7gr6.html#c6b14bc329


Checking the data type if the type does not match then write nan

In [ ]:
for row in table_cars.index:
    for col in type_list:
        cel = table_cars.at[row,col]
        try:
            cel = float(cel)
        except:
            table_cars.at[row,col] = np.nan
        

In [ ]:
# #GIT
# Link_ERROR = 0
# Read_announcements = 0
# new_row = {}
# all_data = []
# for page_main in range(1,4):  
#     try:
#         html_data_link = requests.get(f"https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={page_main}")
#         soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
#     except Exception:
#         print("Link_ERROR")
#         continue
#     links = []
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#         if "https://www.otomoto.pl" in offer['href']:
#             links.append(offer['href']) 
#     for page in links:
#         html_data = requests.get(page)
#         soup = BeautifulSoup(html_data.content, 'html5lib')
#         dict_all = take_all(soup)
#         dict_all = data_clean(dict_all)
#         all_data.append(dict_all)
#         Read_announcements += 1
#         clear_output(wait=True)
#         print("Read_announcements ", Read_announcements)
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# delete_duplicates = Read_announcements - len(table_cars.index) 
# print("Delete duplicates: ", delete_duplicates)
# print("Finish")

Adding the file name and database

In [ ]:
table_name = 'Otomoto_full'

In [ ]:
table_cars.to_csv (table_name +'.csv', index = False, header=True)

Connecting to the database

In [ ]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [ ]:
cur = psql.cursor()

Deleting an existing table and creating a new table in the database

In [ ]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + CREATE_TABLE 
cur.execute(createTableCommand)
psql.commit()

Loop that loads data into the database

In [ ]:
a = 0
for row in table_cars.itertuples(index=False):
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  
        
    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+') ON CONFLICT DO NOTHING;'
    insertDataCommand = full_fit+"\r\n"+Value_data
    
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")